In [1]:
from Module_diagnostic_functions import *
from Module_propagation_functions import *


In [4]:
from Module_diagnostic_functions import *
from Module_propagation_functions import *

def main_HRM(varParam_name, z_scaling=10.0, if_close=0, dir_plot=None, if_log=1, if_slice=1, i_start=0):
    time_stamp=time()

    # initialization
    with open(varParam_name, 'rb') as f:
        varParam = pickle.load(f)
    v = srwl_bl.srwl_uti_parse_options(varParam, use_sys_argv=True)
    v.si = True
    srwl_bl.SRWLBeamline(_name=v.name).calc_all(v)

    # incident beam
    wfr = v.w_res; i_plot = 1
    plot_wfr_diagnostic(wfr, label='input', dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)
    srwlpy.SetRepresElecField(wfr, 'f')
    
    t0 = time(); print('Resizing in frequency domain: ', end='')
    srwlpy.ResizeElecField(wfr, 'f', [0, 1., z_scaling]); print('done in', round(time() - t0, 3), 's')

    # diagnostics - input
    diagnostics, diagnostics_names = diagnose_input(wfr)

    # CC1
    label = 'before C1'; print('Propagating to {}: '.format(label), end=''); i_plot+=1; t0 = time()
    bl = set_optics_CRL0_before_C1(v); srwlpy.PropagElecField(wfr, bl); print('done in', round(time() - t0, 3), 's\n')
    plot_wfr_diagnostic(wfr, label=label, dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)

#     label = 'before C2'; print('Propagating to {}: '.format(label), end=''); i_plot+=1; t0 = time()
#     bl = set_optics_C1_before_C2(v); srwlpy.PropagElecField(wfr, bl); print('done in', round(time() - t0, 3), 's\n')
#     plot_wfr_diagnostic(wfr, label=label, dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)

#     label = 'before CRL1'; print('Propagating to {}: '.format(label), end=''); i_plot+=1; t0 = time()
#     bl = set_optics_C2_before_CRL1(v); srwlpy.PropagElecField(wfr, bl); print('done in', round(time() - t0, 3), 's\n')
#     plot_wfr_diagnostic(wfr, label=label, dir_plot=dir_plot, i=i_plot, if_log=if_log, if_slice=if_slice, i_start=i_start)

    # diagnostics - output
    diagnostics, diagnostics_names = diagnose_output(wfr, diagnostics=diagnostics, diagnostics_names=diagnostics_names)

    # I/O
    if if_close == 1:
        dir_diagnostics = dir_plot+'diagnostics_closed/'; mkdir(dir_diagnostics)
    else:
        dir_diagnostics = dir_plot+'diagnostics_open/'; mkdir(dir_diagnostics)
    save_diagnostics(dir_diagnostics, diagnostics, diagnostics_names)

    print('\nbeam diagnostics written to {}'.format(dir_diagnostics))



In [5]:
if __name__ == '__main__':
    main_HRM('varParam0.pkl', z_scaling=6.0, if_close=0, dir_plot='output/HRM/400.0fs/4000.0fs_100.0meV_job0/', if_log=1, if_slice=0, i_start=10)

Gaussian beam electric field calculation ... completed (lasted 0.054 s)
Extracting intensity and saving it to a file ... completed (lasted 1.964 s)
plot skipped
Resizing in frequency domain: done in 1.33 s
fitting for beam diagnostics
done in 0.299 s

Propagating to before C1: Optical Element Setup: CRL Focal Length: 290.0 m
done in 5.092 s

plot skipped
fitting for beam diagnostics
cent_E_out[eV] 9481.000076479932 bw_out[eV] 0.004533508888580196
dur_out[fs] 399
ptilt_x_out[fs_um] 1.4831309370663764e-14
done in 3.041 s


beam diagnostics written to output/HRM/400.0fs/4000.0fs_100.0meV_job0/diagnostics_open/


C:\Users\nanw0\GoogleDrive\SLAC\SLAC-Diling\11. 21 Winter\DXS optics\NERSC\Parameter_scan\test\Module_diagnostic_functions.py:605: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if values == 'Fit failed':


In [10]:
dir_open='output/HRM/400.0fs/4000.0fs_100.0meV_job0/diagnostics_open/'

diagnostic_names_open =  ['6_dur_out[fs]', '7_ptilt_x_out[fs_um]']
diagnostic_names_close = ['0_axis_ev_in[eV]', '1_int_ev_in[a.u.]', '2_axis_ev_out[eV]', '3_int_ev_out[a.u.]', '4_cent_E_out[eV]', '5_bw_out[eV]' ]

diagnostics_open = []
diagnostics_close = []

for name in diagnostic_names_open:
    file_name = dir_open + name + '.out'
    if not os.path.exists(file_name):
        print('file does not exist', file_name)
    else:
        diagnostic = np.loadtxt(file_name)
        diagnostics_open.append(diagnostic)

for name in diagnostic_names_close:
    file_name = dir_open + name + '.out'
    if not os.path.exists(file_name):
        print('file does not exist', file_name)
    else:
        diagnostic = np.loadtxt(file_name)
        diagnostics_close.append(diagnostic)
